In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import requests
import re
import validators
from googletrans import Translator
from translate import Translator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("Datathon_TrainData_org.csv")
data.head()

In [ ]:
x_train = data["Text_of_Speech"]
y_train = data["Emotion"]

In [ ]:
null_value_index = x_train.loc[x_train.isnull()].index
print(null_value_index)

In [ ]:
x_train.drop(null_value_index, axis=0,inplace = True)
y_train.drop(null_value_index, axis=0,inplace = True)

In [ ]:
null_value_index = y_train.loc[y_train.isnull()].index
print(null_value_index)

In [ ]:
x_train.drop(null_value_index, axis=0,inplace = True)
y_train.drop(null_value_index, axis=0,inplace = True)

In [ ]:
y_train.isnull().sum()

In [ ]:
stop_words = stopwords.words("english")

In [ ]:
def remove_un(text):
    te = ""
    for i in text.split("\n"):
        if len(i) > 0:
            te = te + i 
    return te

In [ ]:
def remove_un1(text):
    te = ""
    for i in text.split("\t"):
        if len(i) > 0:
            te = te + i 
    return te

In [ ]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [ ]:
x_train = x_train.apply(remove_un)
x_train = x_train.apply(remove_un1)
x_train = x_train.apply(remove_numbers)

In [ ]:
lem = WordNetLemmatizer()

In [ ]:
def removing_stop_words(text):
    data = nltk.word_tokenize(text)
    te = ""
    for i in data:
        if i.strip().lower() not in stop_words:
            new = lem.lemmatize(i)
            te = te + new + " "
    return te

In [ ]:
x_train = x_train.apply(removing_stop_words)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size = 0.25,random_state = 42)

In [ ]:
x_train_list = speeches = x_train.tolist()
bag_of_words = [nltk.word_tokenize(speech.lower()) for speech in x_train_list]

In [ ]:
x_test_list = speeches = x_test.tolist()
bag_of_words2 = [nltk.word_tokenize(speech.lower()) for speech in x_test_list]

In [ ]:
word2vec_model = Word2Vec(sentences=bag_of_words, vector_size=100, window=5, min_count=1, sg=0)
word2vec_model.train(bag_of_words, total_examples=len(bag_of_words), epochs=10)
vector = word2vec_model.wv['word']

In [ ]:
word2vec_model2 = Word2Vec(sentences=bag_of_words2, vector_size=100, window=5, min_count=1, sg=0)
word2vec_model.train(bag_of_words2, total_examples=len(bag_of_words2), epochs=10)
vector = word2vec_model.wv['word']

In [ ]:
def aggregate_vectors(sentence, model):
    vector_sum = np.zeros(model.vector_size)
    num_words = 0
    for word in sentence:
        if word in model.wv:
            vector_sum += model.wv[word]
            num_words += 1
    if num_words > 0:
        return vector_sum / num_words
    else:
        return np.zeros(model.vector_size)

In [ ]:
X_train_word2vec = [aggregate_vectors(sentence, word2vec_model) for sentence in bag_of_words ]
X_train_word2vec_matrix = np.vstack(X_train_word2vec)

In [ ]:
X_train_word2vec = [aggregate_vectors(sentence, word2vec_model2) for sentence in bag_of_words2]
X_test_word2vec_matrix = np.vstack(X_train_word2vec)

In [ ]:
print(len(X_train_word2vec_matrix))

In [ ]:
vectorizer = TfidfVectorizer()
X_train_count_matrix=vectorizer.fit_transform(x_train)
# X_test_count_matrix = vectorizer.transform(x_test)

#### logistic regression

In [ ]:
logic = LogisticRegression()
logic.fit(X_train_count_matrix,y_train)
print(logic.score(X_train_count_matrix,y_train))
# y_pred = logic.predict(X_test_count_matrix)
# print("accuracy = ",accuracy_score(y_test,y_pred))

#### using the word to vector modeel

In [ ]:
logic = LogisticRegression(max_iter=1000)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_word2vec_matrix)

logic.fit(X_train_scaled ,y_train)
# print(logic.score(X_train_count_matrix,y_train))
y_pred = logic.predict(X_test_word2vec_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

#### SVC

In [ ]:
svc = SVC(probability=True)
svc.fit(X_train_count_matrix,y_train)
# print(logic.score(X_train_count_matrix,y_train))
y_pred = svc.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

#### decision Tree

In [ ]:
dec = DecisionTreeClassifier()
dec.fit(X_train_count_matrix,y_train)
y_pred = dec.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

#### random forest

In [ ]:
ran = RandomForestClassifier()
ran.fit(X_train_count_matrix,y_train)
y_pred = ran.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

#### k nearest nighbours

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_count_matrix,y_train)
y_pred = knn.predict(X_test_count_matrix)
print("accuracy = ",accuracy_score(y_test,y_pred))

In [ ]:
final = VotingClassifier(estimators=[('logic', LogisticRegression()), ('SVC',SVC(probability=True)),("ran",RandomForestClassifier())], voting='soft').fit(X_train_count_matrix, y_train)

In [ ]:
ypred4 = final.predict(X_test_count_matrix)

In [ ]:
accuracy_score(y_test,ypred4)

In [2]:
print(accuracy_score)

0.5391498881431767
